In [28]:
import torch
from torch import nn

from Dataset import Image_dataset
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch.utils.data import WeightedRandomSampler

from Critic import Discriminator
import torch.optim as optim
from PIL import Image
import numpy as np
from torchvision import transforms

from config import *

In [2]:
ds = Image_dataset("Dataset")

labels = []
for i in range(len(ds)):
    _, label = ds[i]
    labels.append(ds[i][1])

class_counts = Counter(labels)
total_samples = len(labels)

weight_per_class = {cls: total_samples/count for cls, count in class_counts.items()}
weights = [weight_per_class[label] for label in labels]

sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
dl = DataLoader(ds, batch_size = 5, sampler=sampler)


In [3]:
critic = Discriminator(in_channels=3)
optimizer = optim.Adam(critic.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()

In [7]:
for epoch in range(EPOCHS):
    batch_loss = 0
    for data in dl:

        x = data[0].to(DEVICE)
        x = x.float()
        critic_output = critic(x.permute(0, -1, 1, 2))
        logits = critic_output.view(critic_output.shape[0], -1).mean(1)

        labels = data[1]
        targets = torch.tensor(
            [1.0 if label == "Not Defective" else 0.0 for label in labels],
            dtype=torch.float32,
            device=critic_output.device
        )
        
        loss = BCE(logits, targets)
        batch_loss += loss.item()
        critic.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f"Epoch: {epoch} | Batch Loss: {batch_loss/len(dl):.4f}")

Epoch: 0 | Batch Loss: 0.3802
Epoch: 1 | Batch Loss: 0.3854
Epoch: 2 | Batch Loss: 0.1661


In [ ]:
image_path = "/Users/mohamedmafaz/Desktop/CountAI/Dataset/50.jpg"

transform = transforms.Compose([
    transforms.Resize((256, 256)),  
    transforms.ToTensor(),          
])

image = Image.open(image_path).convert("RGB")  
tensor_image = transform(image).unsqueeze(0)   

if critic(tensor_image).view(tensor_image.shape[0], -1).mean(1) > 0.5:
    print("Not Defective")
else:
    print("Defective")

Defective
